In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

In [2]:
data_df = pd.read_csv('data_3/dataset_3.csv')
data_df

,Unnamed: 0,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,...,wBPI,lBPI,wp,BPI_diff,ts_diff,RPI_rank_diff,tb_diff,tp_diff,efgp_diff,atr_diff
0,0,2003,10,1104,68,1328,62,N,0,27,...,1.860963,4.187500,0.607143,-2.326537,-23.0,26.0,-7.0,-195.0,-17.166109,-0.289275
1,1,2003,10,1272,70,1393,63,N,0,26,...,3.014493,2.208333,0.793103,0.806159,-27.0,8.0,-64.0,-162.0,-0.256635,0.106266
2,2,2003,11,1266,73,1437,61,N,0,24,...,3.313043,2.100000,0.821429,1.213043,-57.0,-54.0,0.0,29.0,23.577809,0.387663
3,3,2003,11,1296,56,1457,50,N,0,18,...,-1.304622,-0.083333,0.548387,-1.221289,23.0,-68.0,-39.0,214.0,-2.135935,-0.115245
4,4,2003,11,1400,77,1208,71,N,0,30,...,3.992424,2.273026,0.785714,1.719398,-27.0,-3.0,-10.0,70.0,3.355831,-0.471495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61226,61226,2013,146,1257,85,1181,63,N,0,29,...,7.400000,3.474074,0.852941,3.925926,156.0,2.0,23.0,-4.0,-1.623229,-0.186968
61227,61227,2013,146,1276,79,1196,59,N,0,30,...,4.594286,9.609890,0.781250,-5.015604,-37.0,12.0,-18.0,16.0,-2.848441,0.199778
61228,61228,2013,152,1257,72,1455,68,N,0,22,...,7.400000,3.504808,0.852941,3.895192,111.0,-31.0,-8.0,140.0,5.349284,0.105134
61229,61229,2013,152,1276,61,1393,56,N,0,21,...,4.594286,4.525641,0.781250,0.068645,-118.0,3.0,-126.0,-117.0,5.874602,0.380620


In [3]:
data_df['season'].drop_duplicates()

0        2003
4616     2004
9187     2005
13862    2006
18619    2007
23662    2008
28825    2009
34074    2010
39337    2011
44583    2012
49836    2013
55156    2014
Name: season, dtype: int64

In [4]:
tmp_data_df = data_df[['season', 'RPI_rank_diff', 'BPI_diff', 'ts_diff', 'tp_diff', 'tb_diff', 'efgp_diff', 'atr_diff']].copy()

tmp_data_df['expected_result'] = 0
tmp_data_df

,season,RPI_rank_diff,BPI_diff,ts_diff,tp_diff,tb_diff,efgp_diff,atr_diff,expected_result
0,2003,26.0,-2.326537,-23.0,-195.0,-7.0,-17.166109,-0.289275,0
1,2003,8.0,0.806159,-27.0,-162.0,-64.0,-0.256635,0.106266,0
2,2003,-54.0,1.213043,-57.0,29.0,0.0,23.577809,0.387663,0
3,2003,-68.0,-1.221289,23.0,214.0,-39.0,-2.135935,-0.115245,0
4,2003,-3.0,1.719398,-27.0,70.0,-10.0,3.355831,-0.471495,0
...,...,...,...,...,...,...,...,...,...
61226,2013,2.0,3.925926,156.0,-4.0,23.0,-1.623229,-0.186968,0
61227,2013,12.0,-5.015604,-37.0,16.0,-18.0,-2.848441,0.199778,0
61228,2013,-31.0,3.895192,111.0,140.0,-8.0,5.349284,0.105134,0
61229,2013,3.0,0.068645,-118.0,-117.0,-126.0,5.874602,0.380620,0


In [5]:
tmp_tmp_data_df = tmp_data_df * -1
tmp_tmp_data_df['season'] = tmp_tmp_data_df['season'] * -1
tmp_tmp_data_df.pop('expected_result')
tmp_tmp_data_df['expected_result'] = 1

tmp_data_df = pd.concat([tmp_tmp_data_df, tmp_data_df])
tmp_data_df

,season,RPI_rank_diff,BPI_diff,ts_diff,tp_diff,tb_diff,efgp_diff,atr_diff,expected_result
0,2003,-26.0,2.326537,23.0,195.0,7.0,17.166109,0.289275,1
1,2003,-8.0,-0.806159,27.0,162.0,64.0,0.256635,-0.106266,1
2,2003,54.0,-1.213043,57.0,-29.0,-0.0,-23.577809,-0.387663,1
3,2003,68.0,1.221289,-23.0,-214.0,39.0,2.135935,0.115245,1
4,2003,3.0,-1.719398,27.0,-70.0,10.0,-3.355831,0.471495,1
...,...,...,...,...,...,...,...,...,...
61226,2013,2.0,3.925926,156.0,-4.0,23.0,-1.623229,-0.186968,0
61227,2013,12.0,-5.015604,-37.0,16.0,-18.0,-2.848441,0.199778,0
61228,2013,-31.0,3.895192,111.0,140.0,-8.0,5.349284,0.105134,0
61229,2013,3.0,0.068645,-118.0,-117.0,-126.0,5.874602,0.380620,0


In [22]:
train_data_df = tmp_data_df[tmp_data_df['season'] != 2014]
test_data_df = tmp_data_df[tmp_data_df['season'] == 2014]
test_data_df = test_data_df[test_data_df['expected_result'] == 1]

In [23]:
class MyDataset(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        self.data = data

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index) -> tuple:
        row: pd.Series = self.data.iloc[index]

        values = torch.tensor(row.values[1:-1], dtype=torch.float)
        expected_result = torch.tensor(row.values[-1:][0], dtype=torch.long)

        return values, expected_result


In [24]:
train_dataset = MyDataset(train_data_df)
test_dataset = MyDataset(test_data_df)

In [25]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [26]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


In [27]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 7])
Shape of y: torch.Size([64]) torch.int64


In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [35]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=2, bias=True)
  )
)


In [36]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [37]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)

        #print(pred.shape)
        #print(y.shape)

        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        #loss.requires_grad = True
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [38]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.to(dtype=torch.float))
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [39]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.006934  [    0/111738]
loss: 0.612133  [ 6400/111738]
loss: 0.671120  [12800/111738]
loss: 0.523467  [19200/111738]
loss: 0.591063  [25600/111738]
loss: 0.586058  [32000/111738]
loss: 0.479694  [38400/111738]
loss: 0.464790  [44800/111738]
loss: 0.577180  [51200/111738]
loss: 0.453063  [57600/111738]
loss: 0.542241  [64000/111738]
loss: 0.622024  [70400/111738]
loss: 0.506234  [76800/111738]
loss: 0.573769  [83200/111738]
loss: 0.506634  [89600/111738]
loss: 0.491328  [96000/111738]
loss: 0.481723  [102400/111738]
loss: 0.418949  [108800/111738]
Test Error: 
 Accuracy: 75.6%, Avg loss: 0.507594 

Epoch 2
-------------------------------
loss: 0.551412  [    0/111738]
loss: 0.514205  [ 6400/111738]
loss: 0.559901  [12800/111738]
loss: 0.465699  [19200/111738]
loss: 0.416291  [25600/111738]
loss: 0.542596  [32000/111738]
loss: 0.530291  [38400/111738]
loss: 0.580001  [44800/111738]
loss: 0.517706  [51200/111738]
loss: 0.509449  [57600/111738

In [40]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.474874  [    0/111738]
loss: 0.447274  [ 6400/111738]
loss: 0.474687  [12800/111738]
loss: 0.522488  [19200/111738]
loss: 0.384593  [25600/111738]
loss: 0.434253  [32000/111738]
loss: 0.422744  [38400/111738]
loss: 0.423962  [44800/111738]
loss: 0.473639  [51200/111738]
loss: 0.490862  [57600/111738]
loss: 0.449001  [64000/111738]
loss: 0.494202  [70400/111738]
loss: 0.502086  [76800/111738]
loss: 0.525796  [83200/111738]
loss: 0.394448  [89600/111738]
loss: 0.403901  [96000/111738]
loss: 0.474817  [102400/111738]
loss: 0.487775  [108800/111738]
Test Error: 
 Accuracy: 76.1%, Avg loss: 0.495918 

Epoch 2
-------------------------------
loss: 0.480531  [    0/111738]
loss: 0.502428  [ 6400/111738]
loss: 0.347153  [12800/111738]
loss: 0.457078  [19200/111738]
loss: 0.379537  [25600/111738]
loss: 0.484350  [32000/111738]
loss: 0.595830  [38400/111738]
loss: 0.508200  [44800/111738]
loss: 0.515754  [51200/111738]
loss: 0.549271  [57600/111738

In [41]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.682997  [    0/111738]
loss: 0.572428  [ 6400/111738]
loss: 0.406508  [12800/111738]
loss: 0.445010  [19200/111738]
loss: 0.481004  [25600/111738]
loss: 0.615805  [32000/111738]
loss: 0.491615  [38400/111738]
loss: 0.594742  [44800/111738]
loss: 0.562126  [51200/111738]
loss: 0.644676  [57600/111738]
loss: 0.433398  [64000/111738]
loss: 0.529283  [70400/111738]
loss: 0.556310  [76800/111738]
loss: 0.536986  [83200/111738]
loss: 0.427162  [89600/111738]
loss: 0.453057  [96000/111738]
loss: 0.514291  [102400/111738]
loss: 0.550671  [108800/111738]
Test Error: 
 Accuracy: 70.5%, Avg loss: 0.586125 

Epoch 2
-------------------------------
loss: 0.558612  [    0/111738]
loss: 0.515688  [ 6400/111738]
loss: 0.402063  [12800/111738]
loss: 0.521428  [19200/111738]
loss: 0.496118  [25600/111738]
loss: 0.518994  [32000/111738]
loss: 0.605742  [38400/111738]
loss: 0.501688  [44800/111738]
loss: 0.509443  [51200/111738]
loss: 0.452042  [57600/111738

In [42]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.530155  [    0/111738]
loss: 0.474870  [ 6400/111738]
loss: 0.540597  [12800/111738]
loss: 0.553078  [19200/111738]
loss: 0.475195  [25600/111738]
loss: 0.510321  [32000/111738]
loss: 0.510350  [38400/111738]
loss: 0.395886  [44800/111738]
loss: 0.537856  [51200/111738]
loss: 0.528070  [57600/111738]
loss: 0.562530  [64000/111738]
loss: 0.528878  [70400/111738]
loss: 0.443392  [76800/111738]
loss: 0.480321  [83200/111738]
loss: 0.564420  [89600/111738]
loss: 0.441728  [96000/111738]
loss: 0.555528  [102400/111738]
loss: 0.399959  [108800/111738]
Test Error: 
 Accuracy: 75.4%, Avg loss: 0.500554 

Done!
